In [1]:
import pandas as pd
import numpy as np

In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi

dataset = 'blastchar/telco-customer-churn'
path = 'datasets/churn'

api = KaggleApi()
api.authenticate()
api.dataset_download_files(dataset, path=path, unzip=True) 

In [3]:
from glob import glob # recursive wildcard pattern
import pandas as pd

print(glob(f'{path}/*.csv')) # glob(f'{path}/*.csv') : list 형태
print(glob(f'{path}/*.csv')[0])
df = pd.read_csv(glob(f'{path}/*.csv')[0])

['datasets/churn/WA_Fn-UseC_-Telco-Customer-Churn.csv']
datasets/churn/WA_Fn-UseC_-Telco-Customer-Churn.csv


# Part I. 사전 데이터 분석

In [4]:
df.info() # 현재 feature들을 거의 그대로 쓸 예정

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


| Column            | Dtype   | 한글 설명          |
|-------------------|---------|-------------------|
| customerID        | object  | 고객 ID           |
| gender            | object  | 성별              |
| SeniorCitizen     | int64   | 고령자 여부       |
| Partner           | object  | 파트너 여부       |
| Dependents        | object  | 부양 가족 여부    |
| tenure            | int64   | 가입 기간(월) - 시작과 끝을 알 수 없음    |
| PhoneService      | object  | 전화 서비스 가입 여부 |
| MultipleLines     | object  | 다중 회선 사용 여부 |
| InternetService   | object  | 인터넷 서비스 종류 |
| OnlineSecurity    | object  | 온라인 보안 서비스 가입 여부 |
| OnlineBackup      | object  | 온라인 백업 서비스 가입 여부 |
| DeviceProtection  | object  | 기기 보호 서비스 가입 여부 |
| TechSupport       | object  | 기술 지원 서비스 가입 여부 |
| StreamingTV       | object  | TV 스트리밍 서비스 가입 여부 |
| StreamingMovies   | object  | 영화 스트리밍 서비스 가입 여부 |
| Contract          | object  | 계약 유형         |
| PaperlessBilling  | object  | 무페이퍼 청구서 여부 |
| PaymentMethod     | object  | 결제 방법         |
| MonthlyCharges    | float64 | 월 청구 금액      |
| TotalCharges      | object  | 총 청구 금액      |
| Churn             | object  | 이탈 여부  (Y)        |


### 필드 성격의 종류
- 개인정보 및 가족구성
- 서비스 가입 여부
- 계약 형태
- 결제 관련

#### Plotly를 이용한 시각화
- [Plotly](https://plotly.com/python/)

In [6]:
#!pip install plotly # 동적 화면 구성 가능

### 기초데이터 확인

In [5]:
import plotly.express as px

churn_rate = (df['Churn'].value_counts(normalize=True) * 100)
print(churn_rate)
churn_rate_str = churn_rate.map('{:0.1f}%'.format)
print(churn_rate_str) # No, Yes : index, 해당 값 : values

fig = px.bar(churn_rate, x=churn_rate.index, y=churn_rate.values, labels={'y':'비율 (%)', 'index':'Churn'},
             title='고객 이탈 비율', text=churn_rate_str,
             color=churn_rate.index, color_discrete_sequence=px.colors.qualitative.Pastel, template='plotly_white', width=400, height=400)
fig.update_layout(xaxis_title='이탈수', yaxis_title='비율 (%)', showlegend=False)
fig.show()

Churn
No     73.463013
Yes    26.536987
Name: proportion, dtype: float64
Churn
No     73.5%
Yes    26.5%
Name: proportion, dtype: object


In [6]:
# 가입기간별 이탈 비율 시각화(7일 단위)
print(df['tenure'])
df['tenure_group'] = pd.cut(df['tenure'], bins=range(0, 73, 7), right=False) # 7일씩 끊어서 주차별로 확인하기 위해 정리 (bins = range(0,73,7))
print(df['tenure_group'])
# 'tenure_group' 열의 데이터 타입을 문자열로 변환합니다.
df['tenure_group_str'] = df['tenure_group'].apply(lambda x: f'{(x.left // 7) + 1}주차')
# 'Churn' 열의 데이터 타입을 숫자로 변환합니다. 'Yes'는 1로, 'No'는 0으로 매핑합니다.
df['Churn_numeric'] = df['Churn'].map({'Yes': 1, 'No': 0})
# 이제 'Churn_numeric' 열을 사용하여 평균을 계산합니다.
churn_rate = df.groupby('tenure_group_str')['Churn_numeric'].mean() * 100
print(churn_rate)
churn_rate_str = churn_rate.map('{:0.1f}%'.format)
print(churn_rate_str)
fig = px.bar(churn_rate, x=churn_rate.index, y=churn_rate.values, labels={'y':'이탈 비율 (%)', 'index':'고객 잔존 기간'},
                title='가입기간별 고객 이탈 비율', text=churn_rate_str,
                
                color=churn_rate.index, color_discrete_sequence=px.colors.qualitative.Pastel, template='plotly_white', width=800, height=400)
fig.update_layout(xaxis_title='고객 잔존 기간', yaxis_title='이탈 비율 (%)', showlegend=False)
fig.show()


0        1
1       34
2        2
3       45
4        2
        ..
7038    24
7039    72
7040    11
7041     4
7042    66
Name: tenure, Length: 7043, dtype: int64
0         [0.0, 7.0)
1       [28.0, 35.0)
2         [0.0, 7.0)
3       [42.0, 49.0)
4         [0.0, 7.0)
            ...     
7038    [21.0, 28.0)
7039             NaN
7040     [7.0, 14.0)
7041      [0.0, 7.0)
7042    [63.0, 70.0)
Name: tenure_group, Length: 7043, dtype: category
Categories (10, interval[int64, left]): [[0, 7) < [7, 14) < [14, 21) < [21, 28) ... [42, 49) < [49, 56) < [56, 63) < [63, 70)]
tenure_group_str
1주차     52.937205
2주차     35.749386
3주차     30.188679
4주차     23.309609
5주차     22.413793
6주차     20.796460
7주차     16.964286
8주차     15.909091
9주차     11.336032
10주차     9.344262
Name: Churn_numeric, dtype: float64
tenure_group_str
1주차     52.9%
2주차     35.7%
3주차     30.2%
4주차     23.3%
5주차     22.4%
6주차     20.8%
7주차     17.0%
8주차     15.9%
9주차     11.3%
10주차     9.3%
Name: Churn_numeric, dtype: object


/var/folders/s9/fs_hqtf904g1ylb79x0r4y700000gn/T/ipykernel_31376/1999423101.py:10: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### 결과 확인
* 가입기간이 길수록 y값이 row(low?)에 가깝다.

## 결제관련 탐색

In [7]:
# 결제 방법, 월 청구 금액, 총 청구 금액, 페이퍼리스 청구서 여부 4개 필드를 churn 여부에 따라 분포를 2x2로 그래프로 표현

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 2x2 캔버스 생성
fig = make_subplots(rows=2, cols=2, subplot_titles=("결제 방법", "월 청구 금액", "총 청구 금액", "페이퍼리스 청구서 여부"))


# 결제 방법 분포
payment_method_translations ={
    'Electronic check': '전자 결제', 
    'Mailed check': '우편 결제', 
    'Bank transfer (automatic)': '은행 자동이체',
    'Credit card (automatic)': '신용카드 자동결제'
} 
payment_method = df.groupby('Churn')['PaymentMethod'].value_counts(normalize=True).unstack().fillna(0) * 100
print(payment_method)
payment_method = payment_method.iloc[::-1]
print(payment_method)
colors = px.colors.qualitative.Pastel # 컬러 가져오는 부분 - 파스텔톤
for i, method in enumerate(payment_method.columns):
    fig.add_trace(go.Bar(x=payment_method.index.map(lambda x: '이탈' if x == 'Yes' else '유지'), y=payment_method[method], name=payment_method_translations[method], marker_color=colors[i % len(colors)], text=payment_method_translations[method], textposition='auto'), row=1, col=1)


# 월 청구 금액 분포
monthly_charges = df.groupby('Churn')['MonthlyCharges']
fig.add_trace(go.Violin(y=monthly_charges.get_group('Yes'), name='이탈', line_color=colors[0]), row=1, col=2)
fig.add_trace(go.Violin(y=monthly_charges.get_group('No'), name='유지', line_color=colors[1]), row=1, col=2)


# 총 청구 금액 분포
total_charges = df.groupby('Churn')['TotalCharges']
fig.add_trace(go.Box(y=total_charges.get_group('Yes'), name='이탈', marker_color=colors[0]), row=2, col=1)
fig.add_trace(go.Box(y=total_charges.get_group('No'), name='유지', marker_color=colors[1]), row=2, col=1)


# 무페이퍼 청구서 여부 분포
paperless_billing = df.groupby('Churn')['PaperlessBilling'].value_counts(normalize=True).unstack().fillna(0) * 100
paperless_billing = paperless_billing.iloc[::-1]
for i, status in enumerate(paperless_billing.columns):
    fig.add_trace(go.Bar(
        x=paperless_billing.index.map(lambda x: '이탈' if x == 'Yes' else '유지'), 
        y=paperless_billing[status], 
        name="페이퍼리스" if status == 'Yes' else "페이퍼", 
        text=paperless_billing[status].map(lambda x: "{:0.1f}% 페이퍼리스".format(x) if status == 'Yes' else "{:0.1f}% 페이퍼".format(x)),
        marker_color=colors[i % len(colors)]), row=2, col=2)



# 그래프 업데이트
fig.update_layout(height=800, width=1200, title_text="결제관련 탐색")

# 범례 제거
fig.update_layout(showlegend=False)
fig.show()


PaymentMethod  Bank transfer (automatic)  Credit card (automatic)  \
Churn                                                               
No                             24.855044                24.932354   
Yes                            13.804173                12.413055   

PaymentMethod  Electronic check  Mailed check  
Churn                                          
No                    25.009664     25.202938  
Yes                   57.303371     16.479401  
PaymentMethod  Bank transfer (automatic)  Credit card (automatic)  \
Churn                                                               
Yes                            13.804173                12.413055   
No                             24.855044                24.932354   

PaymentMethod  Electronic check  Mailed check  
Churn                                          
Yes                   57.303371     16.479401  
No                    25.009664     25.202938  


### 결제 관련 1차 분석 결과 알게된 사실
* 전자 결제를 하는 사람이 이탈 가능성 월등히 높다. (가설 : 쉬운 탈퇴UX 제공이 이탈을 유도하는 요인일 수 있다.)
* 페이퍼리스 청구서를 받는 사람이 이탈 가능성이 높다. (가설 : 페이퍼리스 청구서는 소비지출 인식을 쉽게 하기 때문에 이탈 가능성이 높다.)
* 월 청구 금액이 높을 수록 이탈 가능성이 높다. (가설: 이미 월청구금액을 낮게 조정한 사람은 계속 이용하고자 하는 의지가 강할 것이다.)
* 조기 이탈에 따른 영향으로 총 청구금액은 이탈자가 당연히 낮게 나타난다.

### 추가로 알아볼 사항
* 결제 방법 및 페이퍼리스 청구서 여부 별로 월 청구금액의 분포
* 결제 방법과 페이퍼리스 청구서 수신 여부의 상관관계
* 각 항목의 범주형 통계가 유의미한지 확인

In [8]:
# 결제 방법 별로 월 청구금액의 분포 및 페이퍼리스 청구서 여부별 월 청구금액의 분포
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, subplot_titles=("결제 방법 별 월 청구금액 분포", "페이퍼리스 청구서 여부별 월 청구금액 분포", "페이퍼리스 청구서 여부와 결제 방법의 밀도히트맵"))

colors = px.colors.qualitative.Pastel
for i, method in enumerate(df['PaymentMethod'].unique()):
    fig.add_trace(go.Violin(y=df[df['PaymentMethod'] == method]['MonthlyCharges'], 
                            name=payment_method_translations[method], 
                            line_color=colors[i % len(colors)],
                            points="all",
                            pointpos=0,
                            jitter=0.8,
                            marker=dict(opacity=0.5, size=2)), row=1, col=1)

colors = px.colors.qualitative.Set2
for i, status in enumerate(df['PaperlessBilling'].unique()):
    fig.add_trace(go.Violin(y=df[df['PaperlessBilling'] == status]['MonthlyCharges'], 
                            name="페이퍼리스" if status == 'Yes' else "페이퍼", 
                            line_color=colors[i % len(colors)],
                            points="all",
                            pointpos=0,
                            jitter=0.8,
                            marker=dict(opacity=0.5, size=2)), row=1, col=2)
# density heatmap  
colors = px.colors.sequential.PuBu_r
fig.add_trace(go.Heatmap(x=df['PaymentMethod'], y=df['PaperlessBilling'], z=df['MonthlyCharges'], colorscale=colors), row=1, col=3)
fig.update_xaxes(tickvals=list(payment_method_translations.keys()), ticktext=list(payment_method_translations.values()), row=1, col=3)
fig.update_yaxes(tickvals=['Yes', 'No'], ticktext=['페이퍼리스', '페이퍼'], row=1, col=3)
# 각 셀에 대한 개수를 표시하는 annotation 추가
for i, row in enumerate(df['PaymentMethod'].unique()):
    for j, col in enumerate(df['PaperlessBilling'].unique()):
        count = len(df[(df['PaymentMethod'] == row) & (df['PaperlessBilling'] == col)])
        # 1000 이하는 검은색으로 표시
        font_color = "black" if count <= 1000 else "white"
        fig.add_annotation(x=i, y=j, text=str(count), showarrow=False, font=dict(color=font_color), row=1, col=3)



fig.update_layout(height=600, width=1500, showlegend=False)
fig.show()


In [9]:
# 결제 방법, 페이퍼리스청구서여부의 이탈 여부의 통계적 유의성을 검정
from scipy.stats import chi2_contingency # 독립성 검정 (카이제곱 검정) : 두 범주형 변수의 독립 기각 여부 판별

# 이탈 여부에 따른 특성별 카이제곱 검정
def chi2_test(df, feature):
    crosstab = pd.crosstab(df['Churn'], df[feature])
    chi2, p, dof, expected = chi2_contingency(crosstab)
    return chi2, p

chi_result = chi2_test(df, 'PaymentMethod'), chi2_test(df, 'PaperlessBilling') # 함수를 두개 동시에 돌려, 변수를 한번에 실을 수 있다?

# 카이제곱 검정 결과 출력
for i, feature in enumerate(['PaymentMethod', 'PaperlessBilling']):
    print(f'{feature}에 대한 카이제곱 검정 결과')
    print(f'  chi2: {chi_result[i][0]:.2f}')
    print(f'  p-value: {chi_result[i][1]:.2f}')
    if chi_result[i][1] < 0.05:
        print('  => 유의미한 결과: 이탈 여부에 영향을 미침')
    else:
        print('  => 유의미하지 않은 결과: 이탈 여부에 영향을 미치지 않음')
    print()
    


PaymentMethod에 대한 카이제곱 검정 결과
  chi2: 648.14
  p-value: 0.00
  => 유의미한 결과: 이탈 여부에 영향을 미침

PaperlessBilling에 대한 카이제곱 검정 결과
  chi2: 258.28
  p-value: 0.00
  => 유의미한 결과: 이탈 여부에 영향을 미침



In [10]:
# 결제 방법, 페이퍼리스청구서여부 별로 월 청구금액 분포에 유의한 차이가 있는지 t-검정
from scipy.stats import ttest_ind # 한 쪽이 연속형일 때, T-test 진행

# 두 집단 간의 평균 차이를 검정하는 t-검정
def t_test(df, feature):
    group1 = df[df[feature] == df[feature].unique()[0]]['MonthlyCharges']
    group2 = df[df[feature] == df[feature].unique()[1]]['MonthlyCharges']
    t, p = ttest_ind(group1, group2)
    return t, p

t_result = t_test(df, 'PaymentMethod'), t_test(df, 'PaperlessBilling')

# t-검정 결과 출력

for i, feature in enumerate(['PaymentMethod', 'PaperlessBilling']):
    print(f'{feature}에 대한 t-검정 결과')
    print(f'  t: {t_result[i][0]:.2f}')
    print(f'  p-value: {t_result[i][1]:.2f}')
    if t_result[i][1] < 0.05:
        print('  => 유의미한 결과: 월 청구금액에 차이가 존재함')
    else:
        print('  => 유의미하지 않은 결과: 월 청구금액에 차이가 존재하지 않음')
    print()
    

PaymentMethod에 대한 t-검정 결과
  t: 40.06
  p-value: 0.00
  => 유의미한 결과: 월 청구금액에 차이가 존재함

PaperlessBilling에 대한 t-검정 결과
  t: 31.57
  p-value: 0.00
  => 유의미한 결과: 월 청구금액에 차이가 존재함



### 결제 관련 2차 분석 결과 알게된 사실
* 전자결제와 페이퍼리스청구여부는 상관성이 높다 (추정 : 전자결제는 UX에서 페이퍼리스청구를 디폴트로 제공하는것으로 보인다. 비즈니스적 인사이트는 없음)
* 전자결제와 은행/카드 자동이체는 월결제금액이 다같이 높지만, 전자결제는 왜도가 높다. (가설 : 전자결제는 결제금액이 높은 상품을 선호하는 경향이 있을 수 있다.)
* 월결제금액의 분포로 봤을때, 낮은 결제금액상품과 높은 결제금액상품으로 이원화된 상품이 존재하는것으로 보인다. (추가확인해볼수 있는 사항 : 결제방법과 페이퍼리스청구 매트릭스별로 월청구금액의 분포)

## 가족 구성 탐색

In [11]:
# 성별, 고령자 여부, 파트너 여부, 부양 가족 여부 4개 필드를 churn 여부에 따라 분포를 2x2로 그래프로 표현하되, 각각의 count와 비율로 표기
fig = make_subplots(rows=2, cols=2, subplot_titles=("성별", "고령자 여부", "파트너 여부", "부양 가족 여부"))

fields = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']
titles = ['성별', '고령자 여부', '파트너 여부', '부양 가족 여부']
for i, field in enumerate(fields):
    for value in df[field].unique():
        filtered_df = df[df[field] == value]
        churn_yes = filtered_df[filtered_df['Churn'] == 'Yes']
        churn_no = filtered_df[filtered_df['Churn'] == 'No']
        total = len(churn_yes) + len(churn_no)
        yes_percentage = (len(churn_yes) / total) * 100
        no_percentage = (len(churn_no) / total) * 100
        fig.add_trace(go.Bar(x=[f'{value} - 해지'], y=[len(churn_yes)], name='해지', marker_color='crimson', text=f'{yes_percentage:.2f}%'), row=(i//2)+1, col=(i%2)+1)
        fig.add_trace(go.Bar(x=[f'{value} - 유지'], y=[len(churn_no)], name='유지', marker_color='lightskyblue', text=f'{no_percentage:.2f}%'), row=(i//2)+1, col=(i%2)+1)
    fig.update_xaxes(title_text=titles[i], row=(i//2)+1, col=(i%2)+1)
    fig.update_yaxes(title_text="고객 수", row=(i//2)+1, col=(i%2)+1)
fig.update_traces(textposition='outside')
fig.update_layout(barmode='group', title_text='가족 구성에 따른 고객 이탈 및 유지 분포 (수치 및 비율)', height=800, width=1000, showlegend=False)


### 결제 관련 1차 분석 결과 알게된 사실
* 성별과 이탈에는 관계가 없다. (가설: 성별 정보는 정확하지 않을수 있다. 대표가입자의 성별 정보일뿐)
* 나이가 많을 수록 이탈 가능성이 높다. (가설: 나이가 많을수록 가격이나 서비스에 대한 민감도가 높다)
* 파트너가 있는 사람은 이탈 가능성이 낮다. (가설: 연인이 있으면 돈걱정 않하는 경향이 있을수도 있고, 상대방의 취향에 따라 혼자 판단하지 않는다)
* 부양가족이 있는 사람은 이탈 가능성이 낮다. (가설: 가족이 있으면 의사결정이 어려워지고, 가족의 의견을 따르는 경향이 있을수도 있다)

### 추가로 확인해볼사항
* 성별을 제외한 각 항목별 교차분석을 해보자
* 범주형 통계가 유의미한지 확인해보자

In [12]:
# 파트너와 부양가족 여부에 따른 이탈률 분석
fig = make_subplots(rows=1, cols=3, subplot_titles=("파트너 및 부양 가족 여부에 따른 이탈 비율", "부양 가족 및 고령자 여부에 따른 이탈 비율", "고령자 및 파트너 여부에 따른 이탈 비율"))

# 파트너와 부양가족 여부에 따른 이탈률 분석
df['Partner_Dependents'] = df['Partner'].replace({'Yes': '파트너O', 'No': '파트너X'}) + ', ' + df['Dependents'].replace({'Yes': '부양가족O', 'No': '부양가족X'})
partner_dependents = df.groupby('Partner_Dependents')['Churn'].value_counts(normalize=True).unstack().fillna(0) * 100
partner_dependents_str = partner_dependents.map('{:0.1f}%'.format)
fig.add_trace(go.Bar(x=partner_dependents.index, y=partner_dependents['Yes'], text=partner_dependents_str['Yes'], name='파트너 및 부양 가족 여부', marker_color='lightpink'), row=1, col=1)

# 부양가족와 고령자 여부에 따른 이탈률 분석
df['Dependents_SeniorCitizen'] = df['Dependents'].replace({'Yes': '부양가족O', 'No': '부양가족X'}) + ', ' + df['SeniorCitizen'].replace({1: '고령자O', 0: '고령자X'})
dependents_senior_citizen = df.groupby('Dependents_SeniorCitizen')['Churn'].value_counts(normalize=True).unstack().fillna(0) * 100
dependents_senior_citizen_str = dependents_senior_citizen.map('{:0.1f}%'.format)
fig.add_trace(go.Bar(x=dependents_senior_citizen.index, y=dependents_senior_citizen['Yes'], text=dependents_senior_citizen_str['Yes'], name='부양 가족 및 고령자 여부', marker_color='lightblue'), row=1, col=2)

# 고령자 여부와 파트너 여부에 따른 이탈률 분석
df['SeniorCitizen_Partner'] = df['SeniorCitizen'].replace({1: '고령자O', 0: '고령자X'}) + ', ' + df['Partner'].replace({'Yes': '파트너O', 'No': '파트너X'})
senior_citizen_partner = df.groupby('SeniorCitizen_Partner')['Churn'].value_counts(normalize=True).unstack().fillna(0) * 100
senior_citizen_partner_str = senior_citizen_partner.map('{:0.1f}%'.format)
fig.add_trace(go.Bar(x=senior_citizen_partner.index, y=senior_citizen_partner['Yes'], text=senior_citizen_partner_str['Yes'], name='고령자 및 파트너 여부', marker_color='lightgreen'), row=1, col=3)

fig.update_layout(title_text='가족 구성에 따른 고객 이탈 비율 분석', showlegend=False, height=400, width=1500)
fig.show()


In [13]:
# 카이제곱 검정을 통해 파트너, 부양가족, 고령자 여부가 이탈 여부에 영향을 미치는지 확인
from scipy.stats import chi2_contingency

fields = ['Partner', 'Dependents', 'SeniorCitizen']
results = []
for field in fields:
    crosstab = pd.crosstab(df[field], df['Churn'])
    chi2, p, dof, expected = chi2_contingency(crosstab)
    results.append([field, chi2, p, dof, expected])
results_df = pd.DataFrame(results, columns=['필드', '카이제곱 통계량', 'p-value', '자유도', '기대값'])
results_df['결과'] = results_df['p-value'].apply(lambda x: '유의함' if x < 0.05 else '유의하지 않음')
results_df


,필드,카이제곱 통계량,p-value,자유도,기대값,결과
0,Partner,158.733382,2.139911e-36,1,"[[2674.7883004401533, 966.2116995598467], [249...",유의함
1,Dependents,189.129249,4.924922e-43,1,"[[3623.9304273746984, 1309.0695726253018], [15...",유의함
2,SeniorCitizen,159.426300,1.510067e-36,1,"[[4335.0523924464005, 1565.9476075535993], [83...",유의함


### 결제 관련 2차 분석 결과 알게된 사실
* 파트너 여부에 따른 이탈율 저하보다는 부양가족 여부에 따른 이탈율 저하가 더 크다.  (가설: 함께 사는 사람이 많을수록 이탈하기 쉽지 않다. 가족상품 패키지가 효과가 있을수 있다)

## 서비스관련 탐색

| PhoneService      | object  | 전화 서비스 가입 여부 |
| OnlineSecurity    | object  | 온라인 보안 서비스 가입 여부 |
| OnlineBackup      | object  | 온라인 백업 서비스 가입 여부 |
| DeviceProtection  | object  | 기기 보호 서비스 가입 여부 |
| TechSupport       | object  | 기술 지원 서비스 가입 여부 |
| StreamingTV       | object  | TV 스트리밍 서비스 가입 여부 |
| StreamingMovies   | object  | 영화 스트리밍 서비스 가입 여부 |

In [41]:
# 전화, 온라인 보안, 온라인 백업, 기기 보호, 기술지원, 스트리밍 TV, 스트리밍 영화 7개 필드를 churn 여부에 따라 분포를 2x4로 그래프로 표현
fig = make_subplots(rows=2, cols=4, subplot_titles=("전화 서비스", "온라인 보안", "온라인 백업", "기기 보호", "기술 지원", "스트리밍 TV", "스트리밍 영화"))

fields = ['PhoneService'
          , 'OnlineSecurity'
          , 'OnlineBackup'
          , 'DeviceProtection'
          , 'TechSupport'
          , 'StreamingTV'
          , 'StreamingMovies'
          ]
titles = ['전화 서비스'
          , '온라인 보안'
          , '온라인 백업'
          , '기기 보호'
          , '기술 지원'
          , '스트리밍 TV'
          , '스트리밍 영화']

for i, field in enumerate(fields):
    for value in df[field].unique():
        filtered_df = df[df[field] == value]
        churn_yes = filtered_df[filtered_df['Churn'] == 'Yes']
        churn_no = filtered_df[filtered_df['Churn'] == 'No']
        total = len(churn_yes) + len(churn_no)
        yes_percentage = (len(churn_yes) / total) * 100
        no_percentage = (len(churn_no) / total) * 100
        fig.add_trace(go.Bar(x=[f'{value} - 해지'], y=[len(churn_yes)], name='해지', marker_color='crimson', text=f'{yes_percentage:.2f}%'), row=(i//4)+1, col=(i%4)+1)
        fig.add_trace(go.Bar(x=[f'{value} - 유지'], y=[len(churn_no)], name='유지', marker_color='lightskyblue', text=f'{no_percentage:.2f}%'), row=(i//4)+1, col=(i%4)+1)
    fig.update_xaxes(title_text=titles[i], row=(i//4)+1, col=(i%4)+1)
    fig.update_yaxes(title_text="고객 수", row=(i//4)+1, col=(i%4)+1)
fig.update_traces(textposition='outside')
fig.update_layout(barmode='group', title_text='부가 서비스에 따른 고객 이탈 및 유지 분포 (수치 및 비율)', height=800, width=1500, showlegend=False)

fig.show()

### 결제 관련 1차 분석 결과 알게된 사실
* 온라인 보안, 온라인백업, 기기보호, 기술지원 4가지 서비스에 가입한 사람은 이탈 가능성이 낮다. (가설: 애당초 서비스를 추가 신청했다는 사실은 서비스에 만족을 느끼고 있는것으로 해석할수 있다.)
* 추가적인 스트리밍 서비스는 이탈율에 영향을 많이 미치지 않는다. (가설: 스트리밍 서비스는 만족에 의한 가입이 아니라, 패키지 구매였을수 있다.)

## 기타사항 탐색

### 과제 제출 방법
- ipynb 파일을 html로 변환하여 제출해주세요
- e-campus 과제 제출란에 업로드해주세요

### [ #과제1 ]
- 가입기간과 총요금, 월요금을 범주형으로 변환하고, 서비스 가입여부 지표들과 교차분석을 진행해보세요. (가입기간 * 월요금이 총요금이 아닐 수 있다)
- 로열티 높은 고객을 만들기 위한 인사이트를 서술해주세요.
- 마크다운셀을 만들어 내용을 기술해주세요.

In [15]:
df['MonthlyCharges'].describe()

count    7043.000000
mean       64.761692
std        30.090047
min        18.250000
25%        35.500000
50%        70.350000
75%        89.850000
max       118.750000
Name: MonthlyCharges, dtype: float64

In [17]:
df['TotalCharges'].value_counts()

TotalCharges
          11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: count, Length: 6531, dtype: int64

In [24]:
df.info() # 현재 feature들을 거의 그대로 쓸 예정

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   customerID                7043 non-null   object  
 1   gender                    7043 non-null   object  
 2   SeniorCitizen             7043 non-null   int64   
 3   Partner                   7043 non-null   object  
 4   Dependents                7043 non-null   object  
 5   tenure                    7043 non-null   int64   
 6   PhoneService              7043 non-null   object  
 7   MultipleLines             7043 non-null   object  
 8   InternetService           7043 non-null   object  
 9   OnlineSecurity            7043 non-null   object  
 10  OnlineBackup              7043 non-null   object  
 11  DeviceProtection          7043 non-null   object  
 12  TechSupport               7043 non-null   object  
 13  StreamingTV               7043 non-null   object

| Column            | Dtype   | 한글 설명          |
|-------------------|---------|-------------------|
| customerID        | object  | 고객 ID           |
| gender            | object  | 성별              |
| SeniorCitizen     | int64   | 고령자 여부       |
| Partner           | object  | 파트너 여부       |
| Dependents        | object  | 부양 가족 여부    |
| tenure            | int64   | 가입 기간(월) - 시작과 끝을 알 수 없음    |
| PhoneService      | object  | 전화 서비스 가입 여부 |
| MultipleLines     | object  | 다중 회선 사용 여부 |
| InternetService   | object  | 인터넷 서비스 종류 |
| OnlineSecurity    | object  | 온라인 보안 서비스 가입 여부 |
| OnlineBackup      | object  | 온라인 백업 서비스 가입 여부 |
| DeviceProtection  | object  | 기기 보호 서비스 가입 여부 |
| TechSupport       | object  | 기술 지원 서비스 가입 여부 |
| StreamingTV       | object  | TV 스트리밍 서비스 가입 여부 |
| StreamingMovies   | object  | 영화 스트리밍 서비스 가입 여부 |
| Contract          | object  | 계약 유형         |
| PaperlessBilling  | object  | 무페이퍼 청구서 여부 |
| PaymentMethod     | object  | 결제 방법         |
| MonthlyCharges    | float64 | 월 청구 금액      |
| TotalCharges      | object  | 총 청구 금액      |
| Churn             | object  | 이탈 여부  (Y)        |


In [25]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Churn,tenure_group,tenure_group_str,Churn_numeric,Partner_Dependents,Dependents_SeniorCitizen,SeniorCitizen_Partner,tenure_group_2,MonthlyCharges_group,TotalCharges_group
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,"[0, 7)",1주차,0,"파트너O, 부양가족X","부양가족X, 고령자X","고령자X, 파트너O","[0.0, 3.6)","[28.3, 33.325)","[0.0, 434.24)"
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,"[28, 35)",5주차,0,"파트너X, 부양가족X","부양가족X, 고령자X","고령자X, 파트너X","[32.4, 36.0)","[53.425, 58.45)","[1736.96, 2171.2)"
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,Yes,"[0, 7)",1주차,1,"파트너X, 부양가족X","부양가족X, 고령자X","고령자X, 파트너X","[0.0, 3.6)","[53.425, 58.45)","[0.0, 434.24)"
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,"[42, 49)",7주차,0,"파트너X, 부양가족X","부양가족X, 고령자X","고령자X, 파트너X","[43.2, 46.8)","[38.35, 43.375)","[1736.96, 2171.2)"
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,Yes,"[0, 7)",1주차,1,"파트너X, 부양가족X","부양가족X, 고령자X","고령자X, 파트너X","[0.0, 3.6)","[68.5, 73.525)","[0.0, 434.24)"


In [23]:
df['tenure_group_2'] = pd.cut(df['tenure'], 20, right=False) # 7일씩 끊어서 주차별로 확인하기 위해 정리 (bins = range(0,73,7))
df['MonthlyCharges_group'] = pd.cut(df['MonthlyCharges'], 20, right=False)
df['TotalCharges_group'] = pd.cut(df['TotalCharges'].replace(' ', '0').astype('float'), 20, right=False)

In [26]:
for i in range(1,4):
    print(i)

1
2


In [29]:
fields = ['PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
titles = ['전화 서비스', '온라인 보안', '온라인 백업', '기기 보호', '기술 지원', '스트리밍 TV', '스트리밍 영화']
text = ''

for i, feild in enumerate(fields) :
    print(i, feild)

0 PhoneService
1 OnlineSecurity
2 OnlineBackup
3 DeviceProtection
4 TechSupport
5 StreamingTV
6 StreamingMovies


In [32]:
df.groupby('MonthlyCharges_group')[fields].value_counts(normalize=True)

/var/folders/s9/fs_hqtf904g1ylb79x0r4y700000gn/T/ipykernel_31376/3687032689.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



MonthlyCharges_group  PhoneService  OnlineSecurity       OnlineBackup         DeviceProtection     TechSupport          StreamingTV          StreamingMovies    
[18.25, 23.275)       Yes           No internet service  No internet service  No internet service  No internet service  No internet service  No internet service    1.0
                      No            No                   No                   No                   No                   No                   No                     0.0
                      Yes           No internet service  Yes                  Yes                  No                   No                   Yes                    0.0
                                    Yes                  No                   No                   No                   No                   No internet service    0.0
                                                                                                                                             No                     0.0

In [47]:
# 가입기간, 총요금, 월요금 -> 범주형 변환
# 서비스 가입여부 지표들과 교차 분석 (crosstab?)
fig = make_subplots(rows=4, cols=3,subplot_titles=("전화 서비스", "온라인 보안", "온라인 백업", "기기 보호", "기술 지원", "스트리밍 TV", "스트리밍 영화"))

fields = [
    'PhoneService'
    , 'OnlineSecurity'
    , 'OnlineBackup'
    , 'DeviceProtection'
    #, 'TechSupport'
    #, 'StreamingTV'
    #, 'StreamingMovies'
    ]
titles = [
    '전화 서비스'
    , '온라인 보안'
    , '온라인 백업'
    , '기기 보호'
    #, '기술 지원'
    #, '스트리밍 TV'
    #, '스트리밍 영화'
    ]

# 가입기간, 월금액, 총금액 -> 범주형 전환 . 
for i, field in enumerate(fields) :
    # edit - 2024/3/26 night
    # # 파트너와 부양가족 여부에 따른 이탈률 분석
    MonthlyCharges_stat = df.groupby('MonthlyCharges_group')[fields].value_counts(normalize=True).unstack().fillna(0) * 100
    MonthlyCharges_stat_str = MonthlyCharges_stat.map('{:0.1f}%'.format)
    fig.add_trace(go.Bar(x=MonthlyCharges_stat.index, y=MonthlyCharges_stat['Yes'], text=MonthlyCharges_stat_str['Yes'], name=titles[i]+'별 월요금', marker_color='lightpink'), row=i+1, col=1)

        # 부양가족와 고령자 여부에 따른 이탈률 분석
    tenure_group_2_stat = df.groupby('tenure_group_2')[fields].value_counts(normalize=True).unstack().fillna(0) * 100
    tenure_group_2_stat_str = tenure_group_2_stat.map('{:0.1f}%'.format)
    fig.add_trace(go.Bar(x=tenure_group_2_stat.index, y=tenure_group_2_stat['Yes'], text=tenure_group_2_stat_str['Yes'], name=titles[i]+'별 가입기간', marker_color='lightblue'), row=i+1, col=2)

     # 고령자 여부와 파트너 여부에 따른 이탈률 분석
    TotalCharges_group_stat = df.groupby('TotalCharges_group')[fields].value_counts(normalize=True).unstack().fillna(0) * 100
    TotalCharges_group_stat_str = TotalCharges_group_stat.map('{:0.1f}%'.format)
    fig.add_trace(go.Bar(x=TotalCharges_group_stat.index, y=TotalCharges_group_stat['Yes'], text=TotalCharges_group_stat_str['Yes'], name=titles[i]+'별 총요금', marker_color='lightgreen'), row=i+1, col=3)

fig.update_layout(title_text='가족 구성에 따른 고객 이탈 비율 분석', showlegend=False, height=400, width=1500)
fig.show()


/var/folders/s9/fs_hqtf904g1ylb79x0r4y700000gn/T/ipykernel_31376/3282950161.py:28: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/s9/fs_hqtf904g1ylb79x0r4y700000gn/T/ipykernel_31376/3282950161.py:33: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/s9/fs_hqtf904g1ylb79x0r4y700000gn/T/ipykernel_31376/3282950161.py:38: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/s9/fs_hqtf904g1ylb79x0r4

TypeError: Type is not JSON serializable: pandas._libs.interval.Interval

In [22]:
for i, field in enumerate(fields):
    for value in df[field].unique():
        filtered_df = df[df[field] == value]
        churn_yes = filtered_df[filtered_df['Churn'] == 'Yes']
        churn_no = filtered_df[filtered_df['Churn'] == 'No']
        total = len(churn_yes) + len(churn_no)
        yes_percentage = (len(churn_yes) / total) * 100
        no_percentage = (len(churn_no) / total) * 100
        fig.add_trace(go.Bar(x=[f'{value} - 해지'], y=[len(churn_yes)], name='해지', marker_color='crimson', text=f'{yes_percentage:.2f}%'), row=(i//4)+1, col=(i%4)+1)
        fig.add_trace(go.Bar(x=[f'{value} - 유지'], y=[len(churn_no)], name='유지', marker_color='lightskyblue', text=f'{no_percentage:.2f}%'), row=(i//4)+1, col=(i%4)+1)
    fig.update_xaxes(title_text=titles[i], row=(i//4)+1, col=(i%4)+1)
    fig.update_yaxes(title_text="고객 수", row=(i//4)+1, col=(i%4)+1)
fig.update_traces(textposition='outside')
fig.update_layout(barmode='group', title_text='부가 서비스에 따른 고객 이탈 및 유지 분포 (수치 및 비율)', height=800, width=1500, showlegend=False)

fig.show()

### [ #과제2 ]
- MultipleLines ( 다중 회선 사용 여부 ), InternetService ( 인터넷 서비스 종류 ), Contract ( 계약 유형 )
- 3가지 필드에 대한 EDA를 수행하고, 결과를 분석해주세요.
- 시각화 툴은 (plotly, sweetbiz, tableau, pygwalk) 중 선택하여 사용해주세요. (캡처본을 붙여도 됩니다)
- 기초통계량 및 통계검정을 진행해주세요

  

In [23]:
# MultipleLines (다중회선사용여부), InternetService(인터넷서비스종류), Contract(계약유형)


# Part II. AutoML by pyCaret

#### AutoML 특징 및 기능
1) 데이터셋에 대한 전처리를 자동으로 수행
2) 다양한 머신러닝 알고리즘을 활용하여 최적의 모델을 찾아줌
3) 여러 모델을 블렌딩하여 최적의 앙상블 모델을 생성
4) 모델 성능을 평가하고, 모델간 비교를 수행
5) 모델을 저장하고, 로드하여 재사용 가능
6) 모델을 배포할 수 있는 코드를 생성
7) 다양한 시각화 기능을 제공

In [24]:
# pytorch 설치
#!pip3 install torch torchvision

In [25]:
# lightgbm 설치
#!pip install --no-binary lightgbm --config-settings=cmake.define.USE_OPENMP=OFF 'lightgbm>=4.0.0'

In [26]:
# pycaret 설치
#!pip install pycaret

In [27]:
#!pip install scipy==1.11

In [28]:
# 데이터셋을 훈련 데이터와 테스트 데이터로 분리
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [29]:
# pycaret을 사용하여 분류 모델을 학습하기 위한 설정을 자동으로 진행
from pycaret.classification import *
setup_clf = setup(data=train_df, target='Churn')
# ImportError: cannot import name 'interp' from 'scipy' 발생하는 분은 pip scipy==1.11로 설치하고 커널 재시작

,Description,Value
0,Session id,5159
1,Target,Churn
2,Target type,Binary
3,Target mapping,"No: 0, Yes: 1"
4,Original data shape,"(5634, 30)"
5,Transformed data shape,"(5634, 78)"
6,Transformed train set shape,"(3943, 78)"
7,Transformed test set shape,"(1691, 78)"
8,Numeric features,4
9,Categorical features,25


* 출력 값 설명
  
| 파라미터 | 값 | 설명 |
| --- | --- | --- |
| 세션 ID | 3088 | 현재 세션의 고유 식별자입니다. |
| 대상(Target) | Churn | 예측하고자 하는 목표 변수입니다. |
| 대상 유형(Target type) | Binary | 목표 변수가 이진 분류임을 나타냅니다. |
| 대상 매핑(Target mapping) | No: 0, Yes: 1 | 'No'와 'Yes'를 각각 0과 1로 매핑합니다. |
| 원본 데이터 형태(Original data shape) | (5634, 21) | 원본 데이터는 5634개의 샘플과 21개의 특성을 가집니다. |
| 변환된 데이터 형태(Transformed data shape) | (5634, 42) | 전처리 후 데이터는 5634개의 샘플과 42개의 특성을 가집니다. |
| 변환된 훈련 세트 형태(Transformed train set shape) | (3943, 42) | 훈련 데이터는 3943개의 샘플과 42개의 특성을 가집니다. |
| 변환된 테스트 세트 형태(Transformed test set shape) | (1691, 42) | 테스트 데이터는 1691개의 샘플과 42개의 특성을 가집니다. |
| 숫자형 특성(Numeric features) | 3 | 숫자형 특성의 개수입니다. |
| 범주형 특성(Categorical features) | 17 | 범주형 특성의 개수입니다. |
| 전처리(Preprocess) | True | 데이터 전처리가 수행되었음을 나타냅니다. |
| 대입 유형(Imputation type) | simple | 단순 대입 방식을 사용했음을 나타냅니다. |
| 숫자형 대입(Numeric imputation) | mean | 숫자형 데이터의 결측치를 평균값으로 대체합니다. |
| 범주형 대입(Categorical imputation) | mode | 범주형 데이터의 결측치를 최빈값으로 대체합니다. |
| 최대 원-핫 인코딩(Maximum one-hot encoding) | 25 | 원-핫 인코딩을 적용할 최대 범주의 수입니다. |
| 인코딩 방법(Encoding method) | None | 특별한 인코딩 방법을 사용하지 않았음을 나타냅니다. |
| 폴드 생성기(Fold Generator) | StratifiedKFold | 계층화 K-폴드 교차 검증을 사용합니다. |
| 폴드 수(Fold Number) | 10 | 교차 검증에 사용할 폴드의 수입니다. |
| CPU 작업(CPU Jobs) | -1 | 사용 가능한 모든 CPU 코어를 사용합니다. |
| GPU 사용(Use GPU) | False | GPU를 사용하지 않음을 나타냅니다. |
| 실험 기록(Log Experiment) | False | 실험을 기록하지 않음을 나타냅니다. |
| 실험 이름(Experiment Name) | clf-default-name | 실험의 이름입니다. |
| USI | 69aa | 고유 세션 식별자입니다. |



## 모델 비교

| 모델 | 설명 |
| --- | --- |
| SVM | 선형 및 비선형 데이터에 대한 다목적 모델. 데이터가 클때는 시간이 많이 걸림. 이탈과 비 이탈 간의 패턴이 복잡할 때 유용 |
| LogReg | 간단하고 해석이 용이한 모델. 이탈 예측과 같은 이진 분류 작업에 적합. 기능이 이탈 가능성에 어떤 영향을 미치는지 파악에 활용가능. |
| RF | 여러 의사결정 트리를 결합하는 앙상블모델. 범주형 및 숫자형을 결합해 처리 가능. 과적합에 잘 버티고 복잡한 데이터셋에 알맞음. |
| GB | 강력한 예측력. 순차 앙상블 모델. 데이터의 복잡한 관계를 포착. 다양한 유형의 데이터셋에 적합. |
| CatBoost | 범주형 변수를 자동으로 처리하는 그래디언트 부스팅 모델. 과적합을 방지하고 정확도를 높임. 빠른 속도와 높은 정확도를 제공. |
| LightGBM | 빠른 속도와 높은 정확도를 제공하는 그래디언트 부스팅 모델. 대용량 데이터셋에 적합. 과적합을 방지하고 정확도를 높임. |
| Stacking | 여러 모델을 조합하여 최적의 모델을 만드는 앙상블 모델. 다양한 모델을 조합하여 정확도를 높임. |

### 모델 리스트
<img src="https://blog.kakaocdn.net/dn/pvnGP/btrybYDos57/Q5WZrDXsEY5Nwl4WJ92sL1/img.png" height=700>

In [30]:
# 단일 모델 생성하기
lightgbm = create_model('lightgbm', fold=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:34:14
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pycaret/internal/pipeline.py", line 276, in fit
    fitted_estimator = self._memory_fit(
                       ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/joblib/memory.py", line 353, in __call__
    return self.func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Fr

In [ ]:
# 정확도를 기준으로 상위 5개 모델 생성하기
top5 = compare_models(sort='Accuracy', n_select=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.1820
nb,Naive Bayes,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0490
dt,Decision Tree Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0500
ridge,Ridge Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0480
qda,Quadratic Discriminant Analysis,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0570
ada,Ada Boost Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0500
svm,SVM - Linear Kernel,0.9295,0.0000,0.9295,0.9620,0.9204,0.8266,0.8503,0.0510
et,Extra Trees Classifier,0.9061,0.0000,0.9061,0.9183,0.8963,0.7208,0.7538,0.0760
knn,K Neighbors Classifier,0.8179,0.0000,0.8179,0.8116,0.8133,0.5116,0.5144,0.0590
rf,Random Forest Classifier,0.8060,0.0000,0.8060,0.8473,0.7580,0.3446,0.4516,0.0870


In [ ]:
# 상위 5개 모델을 튜닝하기
tuned_top5 = [tune_model(i) for i in top5]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9367,0.0000,0.9367,0.9395,0.9375,0.8422,0.8435
1,0.9266,0.0000,0.9266,0.9268,0.9267,0.8125,0.8125
2,0.9013,0.0000,0.9013,0.9121,0.9039,0.7622,0.7685
3,0.9416,0.0000,0.9416,0.9461,0.9427,0.8563,0.8592
4,0.8655,0.0000,0.8655,0.8750,0.8685,0.6728,0.6765
5,0.9239,0.0000,0.9239,0.9318,0.9256,0.8153,0.8204
6,0.8832,0.0000,0.8832,0.8901,0.8854,0.7135,0.7163
7,0.9036,0.0000,0.9036,0.9057,0.9044,0.7563,0.7569
8,0.9162,0.0000,0.9162,0.9227,0.9179,0.7940,0.7975


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
# 앙상블 모델 생성하기
blender_top5 = blend_models(estimator_list=tuned_top5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7899,0.0000,0.7899,0.7770,0.7647,0.3630,0.3944
1,0.8152,0.0000,0.8152,0.8097,0.7955,0.4477,0.4790
2,0.8025,0.0000,0.8025,0.7966,0.7768,0.3948,0.4348
3,0.7893,0.0000,0.7893,0.7950,0.7462,0.3059,0.3802
4,0.7741,0.0000,0.7741,0.7553,0.7416,0.2947,0.3303
5,0.7893,0.0000,0.7893,0.7993,0.7442,0.3004,0.3811
6,0.7716,0.0000,0.7716,0.7568,0.7309,0.2696,0.3192
7,0.7919,0.0000,0.7919,0.7856,0.7600,0.3493,0.3972
8,0.7944,0.0000,0.7944,0.7836,0.7698,0.3780,0.4108


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# 최종 모델 생성하기
final_model = finalize_model(blender_top5)

In [ ]:
# 테스트 데이터에 대한 예측 결과 생성하기
prediction = predict_model(final_model, data=test_df.drop('Churn', axis=1))

In [ ]:
# 테스트셋에 대한 예측 정확도 확인하기
from pycaret.utils.generic import check_metric
check_metric(test_df['Churn'], prediction['prediction_label'], metric = 'Accuracy')

0.7871

# Part III. CometML 과 Optuna 를 활용한 모델 설계 및 하이퍼파라미터 튜닝

### COMET ML 소개
- MLOps
- COMET ML은 머신러닝 실험을 위한 온라인 플랫폼
- 실험을 추적하고, 실험 결과를 시각화하며, 모델을 관리하는데 사용
- https://www.comet.com/

### optuna 패키지 소개
- Optuna는 하이퍼파라미터 최적화를 위한 오픈소스 라이브러리입니다.
- Optuna는 베이즈 최적화를 기반으로 하이퍼파라미터 최적화를 수행합니다.
- Optuna는 다양한 머신러닝 프레임워크와 호환되며, 사용자가 최적화하고자 하는 함수를 정의하면 Optuna가 하이퍼파라미터 최적화를 수행합니다.
- Optuna는 다양한 알고리즘을 제공하며, 사용자가 원하는 알고리즘을 선택하여 최적화를 수행할 수 있습니다.

In [ ]:
#!pip install comet_ml
#!pip install xgboost
#!pip install optuna

In [ ]:
import optuna

In [ ]:
# XGBClassifier
from xgboost import XGBClassifier
# LightGBM
from lightgbm import LGBMClassifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
# 가입 절차 필요 (https://www.comet.com/)

experiment = Experiment(
  api_key="xxxxxxxxxxx",
  project_name="xxxxxxxxxxx",
  workspace="xxxxxxxxxxx"
)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, xgboost, lightgbm.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: The given API key xxxxxxxxxxx is invalid on www.comet.com, please check it against the dashboard. Your experiment will not be logged 
For more details, please refer to: https://www.comet.com/docs/v2/api-and-sdk/python-sdk/warnings-errors/
COMET ERROR: The given API key xxxxxxxxxxx is invalid on www.comet.com, please check it against the dashboard. Your experiment will not be logged 
For more details, please refer to: https://www.comet.com/docs/v2/api-and-sdk/python-sdk/warnings-errors/


In [ ]:
# 앞서 분석한 내용중, 유의미한 특성을 범주형과 수치형, 그리고 타겟으로 나누어 저장
categorical_features = ['SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'PaperlessBilling']
numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
target_variable = 'Churn'

In [ ]:
# float 타입으로 변환되지 못하는 행을 처리
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')

In [ ]:
# 타겟값 바이너리화
df.Churn = df.Churn.map({'Yes': 1, 'No': 0})

In [ ]:
# 원핫 인코더와 스케일러 준비
encoder = OneHotEncoder(handle_unknown="ignore")
scaler = StandardScaler()

In [ ]:
# 데이터셋 분리
X_train, X_val, y_train, y_val = train_test_split(df.drop("Churn", axis=1), df["Churn"], test_size=0.2, random_state=42)

In [ ]:
# 범주형 인코딩
X_train_encoded = encoder.fit_transform(X_train[categorical_features])
X_val_encoded = encoder.transform(X_val[categorical_features])

In [ ]:
# 수치형 스케일링
X_train_scaled = scaler.fit_transform(X_train[numerical_features])
X_val_scaled = scaler.transform(X_val[numerical_features])

In [ ]:
# 인코딩된 범주형 데이터와 스케일링된 수치형 데이터를 합치기
X_train_processed = np.concatenate((X_train_encoded.toarray(), X_train_scaled), axis=1)
X_val_processed = np.concatenate((X_val_encoded.toarray(), X_val_scaled), axis=1)

In [ ]:
# NaN 값을 평균으로 채우기
X_train_processed = np.nan_to_num(X_train_processed, nan=np.nanmean(X_train_processed))
X_val_processed = np.nan_to_num(X_val_processed, nan=np.nanmean(X_val_processed))

In [ ]:
# 데이터셋 크기 확인
X_train_processed.shape, X_val_processed.shape

((5634, 28), (1409, 28))

#### Optuna 목적함수 작성

In [ ]:
def objective(trial):
    # 하이퍼파라미터 탐색 공간 정의
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 100, 300),
        'max_depth': trial.suggest_categorical('xgb_max_depth', [3, 4, 5]),
        'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.2),
        'min_child_weight': trial.suggest_int('xgb_min_child_weight', 1, 5),
    }

    lgbm_params = {
        'n_estimators': trial.suggest_int('lgbm_n_estimators', 100, 300),
        'max_depth': trial.suggest_categorical('lgbm_max_depth', [3, 4, 5]),
        'learning_rate': trial.suggest_float('lgbm_learning_rate', 0.01, 0.2),
    }

    logreg_params = {
        'C': trial.suggest_categorical('logreg_C', [0.1, 1, 10]),
        'solver': trial.suggest_categorical('logreg_solver', ['liblinear', 'lbfgs']),
    }
    
    # 모델 훈련
    xgb = XGBClassifier()
    lgbm = LGBMClassifier()
    logreg = LogisticRegression()
    
    xgb.fit(X_train_processed, y_train)
    lgbm.fit(X_train_processed, y_train)
    logreg.fit(X_train_processed, y_train)
    
    # 검증 데이터에 대한 예측
    xgb_predictions = xgb.predict(X_val_processed)
    lgbm_predictions = lgbm.predict(X_val_processed)
    logreg_predictions = logreg.predict(X_val_processed)
    
    # 검증 데이터에 대한 예측 정확도 계산
    xgb_accuracy = accuracy_score(y_val, xgb_predictions)
    lgbm_accuracy = accuracy_score(y_val, lgbm_predictions)
    logreg_accuracy = accuracy_score(y_val, logreg_predictions)
    
    # 검증 데이터에 대한 예측 ROC AUC 계산
    xgb_roc_auc = roc_auc_score(y_val, xgb.predict_proba(X_val_processed)[:, 1])
    lgbm_roc_auc = roc_auc_score(y_val, lgbm.predict_proba(X_val_processed)[:, 1])
    logreg_roc_auc = roc_auc_score(y_val, logreg.predict_proba(X_val_processed)[:, 1])

    
    
    # StackingClassifier를 사용하여 앙상블 모델 생성
    estimators = [
        ('xgboost', xgb),
        ('lightgbm', lgbm),
        ('logreg', logreg)
    ]
    stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
    stacking_classifier.fit(X_train_processed, y_train)
    
    # StackingClassifier의 검증 데이터에 대한 예측 정확도 계산
    stacking_predictions = stacking_classifier.predict(X_val_processed)
    stacking_accuracy = accuracy_score(y_val, stacking_predictions)
    stacking_roc_auc = roc_auc_score(y_val, stacking_classifier.predict_proba(X_val_processed)[:, 1])
    
    # Comet ML 로 하이퍼파라미터 및 메트릭 기록
    experiment.log_parameters({
        'xgb_n_estimators': xgb_params['n_estimators'],
        'xgb_max_depth': xgb_params['max_depth'],
        'xgb_learning_rate': xgb_params['learning_rate'],
        'xgb_min_child_weight': xgb_params['min_child_weight'],
        'lgbm_n_estimators': lgbm_params['n_estimators'],
        'lgbm_max_depth': lgbm_params['max_depth'],
        'lgbm_learning_rate': lgbm_params['learning_rate'],
        'logreg_C': logreg_params['C'],
        'logreg_solver': logreg_params['solver']
    })
    
    # 정확도 및 ROC AUC 기록
    experiment.log_metrics({
        'xgb_accuracy': xgb_accuracy,
        'lgbm_accuracy': lgbm_accuracy,
        'logreg_accuracy': logreg_accuracy,
        'xgb_roc_auc': xgb_roc_auc,
        'lgbm_roc_auc': lgbm_roc_auc,
        'logreg_roc_auc': logreg_roc_auc,
        'stacking_accuracy': stacking_accuracy,
        'stacking_roc_auc': stacking_roc_auc
    })
    
    # 목적함수를 최소화하기 위해 음수 정확도를 반환합니다
    return -stacking_accuracy

#### Optuna 최적화 수행

In [ ]:
from tabulate import tabulate

# 연구 생성 및 최적화
study = optuna.create_study(direction='minimize')  # 최적화 목표에 따라 방향 조정
study.optimize(objective, n_trials=10)  # 시도 횟수 지정

# 최적 하이퍼파라미터 및 결과 가져오기
best_params = study.best_params # 최적의 하이퍼파라미터
best_accuracy = -study.best_value  # 양수 정확도로 변환

# 딕셔너리를 테이블 형식의 키-값 쌍 리스트로 변환
param_table = [(key, value) for key, value in best_params.items()]

# 최적 하이퍼파라미터 테이블 표시
best_params = tabulate(param_table, headers=["매개변수", "값"], tablefmt="grid")

print(f"최적의 RF 하이퍼파라미터:\n{best_params}")
print(f"최고점수의 정확도: {best_accuracy}")



[I 2024-03-23 14:39:18,762] A new study created in memory with name: no-name-48dde86f-b935-443b-9975-4fba285c4804


[LightGBM] [Info] Number of positive: 1496, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000336 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265531 -> initscore=-1.017418
[LightGBM] [Info] Start training from score -1.017418
[LightGBM] [Info] Number of positive: 1496, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000323 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 28
[LightGBM] [Info] [binary:

[I 2024-03-23 14:39:20,529] Trial 0 finished with value: -0.8161816891412349 and parameters: {'xgb_n_estimators': 289, 'xgb_max_depth': 4, 'xgb_learning_rate': 0.11310007274185307, 'xgb_min_child_weight': 2, 'lgbm_n_estimators': 108, 'lgbm_max_depth': 3, 'lgbm_learning_rate': 0.17315216409347092, 'logreg_C': 10, 'logreg_solver': 'lbfgs'}. Best is trial 0 with value: -0.8161816891412349.


[LightGBM] [Info] Number of positive: 1496, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000329 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265531 -> initscore=-1.017418
[LightGBM] [Info] Start training from score -1.017418
[LightGBM] [Info] Number of positive: 1496, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000324 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 28
[LightGBM] [Info] [binary:

[I 2024-03-23 14:39:22,194] Trial 1 finished with value: -0.8161816891412349 and parameters: {'xgb_n_estimators': 181, 'xgb_max_depth': 3, 'xgb_learning_rate': 0.11497924137574389, 'xgb_min_child_weight': 2, 'lgbm_n_estimators': 164, 'lgbm_max_depth': 3, 'lgbm_learning_rate': 0.051629408338005896, 'logreg_C': 10, 'logreg_solver': 'lbfgs'}. Best is trial 0 with value: -0.8161816891412349.


...

[LightGBM] [Info] Number of positive: 1496, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000333 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265531 -> initscore=-1.017418
[LightGBM] [Info] Start training from score -1.017418
[LightGBM] [Info] Number of positive: 1496, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000326 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 28
[LightGBM] [Info] [binary:

[I 2024-03-23 14:39:33,691] Trial 8 finished with value: -0.8161816891412349 and parameters: {'xgb_n_estimators': 278, 'xgb_max_depth': 4, 'xgb_learning_rate': 0.03450748354662812, 'xgb_min_child_weight': 1, 'lgbm_n_estimators': 268, 'lgbm_max_depth': 4, 'lgbm_learning_rate': 0.12698875525646755, 'logreg_C': 1, 'logreg_solver': 'liblinear'}. Best is trial 0 with value: -0.8161816891412349.


[LightGBM] [Info] Number of positive: 1496, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000324 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265531 -> initscore=-1.017418
[LightGBM] [Info] Start training from score -1.017418
[LightGBM] [Info] Number of positive: 1496, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000339 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 28
[LightGBM] [Info] [binary:

[I 2024-03-23 14:39:35,112] Trial 9 finished with value: -0.8161816891412349 and parameters: {'xgb_n_estimators': 279, 'xgb_max_depth': 4, 'xgb_learning_rate': 0.06479006304285251, 'xgb_min_child_weight': 4, 'lgbm_n_estimators': 265, 'lgbm_max_depth': 5, 'lgbm_learning_rate': 0.19488134064090432, 'logreg_C': 0.1, 'logreg_solver': 'liblinear'}. Best is trial 0 with value: -0.8161816891412349.


최적의 RF 하이퍼파라미터:
+----------------------+---------------------+
| 매개변수             | 값                  |
+======================+=====================+
| xgb_n_estimators     | 289                 |
+----------------------+---------------------+
| xgb_max_depth        | 4                   |
+----------------------+---------------------+
| xgb_learning_rate    | 0.11310007274185307 |
+----------------------+---------------------+
| xgb_min_child_weight | 2                   |
+----------------------+---------------------+
| lgbm_n_estimators    | 108                 |
+----------------------+---------------------+
| lgbm_max_depth       | 3                   |
+----------------------+---------------------+
| lgbm_learning_rate   | 0.17315216409347092 |
+----------------------+---------------------+
| logreg_C             | 10                  |
+----------------------+---------------------+
| logreg_solver        | lbfgs               |
+----------------------+---------------------+
최고

In [ ]:
# 딕셔너리를 사용하여 여러 하이퍼파라미터 보고:
hyper_params = {
   
   "learning_rate": 0.5,
   "steps": 100000,
   "batch_size": 50,
}
experiment.log_parameters(hyper_params)


#### 결과확인은 COMET ML을 통해 확인